<a href="https://colab.research.google.com/github/returaj/cs6910/blob/assginment1_akash/q10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/DLASSIGNMENT/activation_func.py .
import activation_func
!cp drive/MyDrive/DLASSIGNMENT/loss_func.py .
import loss_func
!cp drive/MyDrive/DLASSIGNMENT/runner.py .
!cp drive/MyDrive/DLASSIGNMENT/feed_forward_nn.py .
!cp drive/MyDrive/DLASSIGNMENT/measure.py .
!cp drive/MyDrive/DLASSIGNMENT/optimizer.py .


Mounted at /content/drive


In [4]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 144 kB 55.7 MB/s 
     |████████████████████████████████| 181 kB 59.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=2c2a29f7ae19eb486c8516a84ec031298d29204b3aa7e20f978fba75377a324e
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [26]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [27]:
import numpy as np
from keras.datasets import mnist

from feed_forward_nn import FNN
from measure import accuracy
from optimizer import SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam
from activation_func import Sigmoid, Relu, Tanh
from loss_func import CrossEntropy, MeanSquaredError
from sklearn.model_selection import train_test_split
import numpy as np
from keras.datasets import mnist
from feed_forward_nn import FNN
from measure import accuracy
from optimizer import SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam
from activation_func import Sigmoid, Relu, Tanh
from loss_func import CrossEntropy, MeanSquaredError
from sklearn.model_selection import train_test_split
#np.random.seed(2)
import wandb

In [38]:
class digits_mnist(object):
  def __init__(self, MnistData=True):
    if MnistData:
      self.initialize_mnist_data()
    else:
      self.initialize_data()

  def initialize_mnist_data(self):
    (X, self.y), (X_test, self.y_test) = mnist.load_data()
    self.X = np.array([x.flatten() for x in X]) / 255
    self.X_test = np.array([x.flatten() for x in X_test]) / 255

  def initialize_data(self):
    raise NotImplementedError("Please implement this method if you need other dataset.")

  @staticmethod
  def get_loss_function(key):
    mapper = {
      "cross_entropy": CrossEntropy,
      "mse": MeanSquaredError
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_activation_function(key):
    mapper = {
      "sigmoid": Sigmoid,
      "relu": Relu,
      "tanh": Tanh
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_optimizer(key):
    mapper = {
      "sgd": SGD,
      "momentum_gd": MomentumGD,
      "nesterov_gd": NesterovGD,
      "rmsprop": Rmsprop,
      "adam": Adam,
      "nadam": Nadam
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def train(X_train, y_train, params, do_val, wandb_log):
    """
    X_train: (batch_size(B), data_size(N))
    y_train: (batch_size(B))
    params: dict(
      batch_size: int,
      epochs: int,
      alpha: float,
      optimizer: One of (SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam),
      hidden_layers_size: list(layer_size),
      act_func: One of (Sigmoid, Relu, Tanh),
      reg: float,
      init: One of (random, xavier),
      loss_func: One of (CrossEntropy, MeanSquaredError)
    )
    """
    if do_val:
      X_train, X_val, y_train, y_val = train_test_split(
          X_train, y_train, test_size=0.1, random_state=10)

    data_size, input_size = X_train.shape
    output_size = 10
    batch_size, epochs = params['batch_size'], params['epochs']
    act_func, loss_func = params['act_func'](), params['loss_func']()
    model = FNN(
      input_size         = input_size,
      output_size        = output_size,
      hidden_layers_size = params['hidden_layers_size'],
      act_func           = act_func,
      reg                = params['reg'],
      init               = params['init'],
      loss_func          = loss_func,
    )
    optimizer = params['optimizer'](model, params['alpha'])
    for ep in range(1, epochs+1):
      ids = np.arange(data_size)
      np.random.shuffle(ids)
      start, end = 0, batch_size
      while end > start:
        x, y = X_train[ids[start:end]], y_train[ids[start:end]]
        optimizer.optimize(x, y)
        start, end = end, min(data_size, end+batch_size)
      # log
      train_loss = loss_func.error(X_train, y_train, model)
      estimate_y_train = digits_mnist.predict(X_train, model)
      train_acc = accuracy(estimate_y_train, y_train)
      val_loss, val_acc = "NotDefined", "NotDefined"
      if do_val:
        val_loss = loss_func.error(X_val, y_val, model)
        estimate_y_val = digits_mnist.predict(X_val, model)
        val_acc = accuracy(estimate_y_val, y_val)
      digits_mnist.logger(train_loss, train_acc, val_loss, val_acc, ep, wandb_log)
    return model

  def logger(train_loss, train_acc, val_loss, val_acc, step, wandb_log):
    # print(f"TrainingLoss: {train_loss}, TrainingAccuracy: {train_acc}")
    # print(f"ValidationLoss: {val_loss}, ValidationAccuracy: {val_acc}")
    if wandb_log:
      wandb.log({
        "epoch": step,
        "TrainingLoss": train_loss,
        "TrainingAccuracy": train_acc,
        "ValidationLoss": val_loss,
        "ValidationAccuracy": val_acc,  
      })

  @staticmethod
  def predict(X, model):
    """
    X: (batch_size(B), data_size(N))
    return np.array of size: (batch_size(B),)
    """
    prob = model.forward(X)[-1]
    return np.argmax(prob, axis=1)


WANDB_PROJECT = "mnist"
WANDB_ENTITY  = "cs21m003_cs21d406"


def run_wandb():
  wandb.init(project=WANDB_PROJECT, entity=WANDB_ENTITY)
  config = wandb.config
  loss_name = "ce" if config.loss_func == "cross_entropy" else "mse"
  wandb.run.name=f"e_{config.epochs}_bs_{config.batch_size}_hl_{config.hidden_layers}_hn_{config.hidden_nodes}_init_{config.weight_init}_ac_{config.act_func}_reg_{config.reg}_ls_{loss_name}_opt_{config.optimizer}_lr_{config.learning_rate}_sc_{config.search_type}_mnist_best"
  hidden_layers_size = [config.hidden_nodes] * config.hidden_layers
  runner = digits_mnist()
  params = {
    "batch_size"        : config.batch_size,
    "epochs"            : config.epochs,
    "alpha"             : config.learning_rate,
    "optimizer"         : runner.get_optimizer(config.optimizer),
    "hidden_layers_size": hidden_layers_size,
    "act_func"          : runner.get_activation_function(config.act_func),
    "reg"               : config.reg,
    "init"              : config.weight_init,
    "loss_func"         : runner.get_loss_function(config.loss_func),
  }
  model = runner.train(runner.X, runner.y, params,do_val=True, wandb_log=True)
  estimate_y_test = runner.predict(runner.X_test, model)
  test_acc = accuracy(estimate_y_test, runner.y_test)
  wandb.log({"TestAccuracy": test_acc})


def do_hyperparameter_search_using_wandb(search_type, loss_func, cnt=5):
  sweep_config_1={
    "name": f"{search_type} sweep",
    "method": search_type,
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "epochs": {"values": [5]}, 
      "batch_size": {"values": [ 32]}, 
      "hidden_layers": {"values": [3]}, 
      "hidden_nodes": {"values": [128]},
      "reg": {"values": [0]},
      "weight_init": {"values": ['xavier']} , 
      "act_func": {"values": ["tanh"]}, 
      "loss_func": {"values": [loss_func]}, 
      "learning_rate": {"values": [1e-3]},   
      "optimizer": {"values": ["adam"]},
      "search_type": {"values": [search_type]}
    }
  }
  sweep_config_2={
    "name": f"{search_type} sweep",
    "method": search_type,
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "epochs": {"values": [5]}, 
      "batch_size": {"values": [64]}, 
      "hidden_layers": {"values": [3]}, 
      "hidden_nodes": {"values": [128]},
      "reg": {"values": [0]},
      "weight_init": {"values": ['xavier']} , 
      "act_func": {"values": ["tanh"]}, 
      "loss_func": {"values": [loss_func]}, 
      "learning_rate": {"values": [1e-3]},   
      "optimizer": {"values": ["adam"]},
      "search_type": {"values": [search_type]}
    }
  }
  sweep_config_3={
    "name": f"{search_type} sweep",
    "method": search_type,
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "epochs": {"values": [5]}, 
      "batch_size": {"values": [ 32]}, 
      "hidden_layers": {"values": [3]}, 
      "hidden_nodes": {"values": [32]},
      "reg": {"values": [0.0005]},
      "weight_init": {"values": ['xavier']} , 
      "act_func": {"values": ["tanh"]}, 
      "loss_func": {"values": [loss_func]}, 
      "learning_rate": {"values": [1e-3]},   
      "optimizer": {"values": ["nadam"]},
      "search_type": {"values": [search_type]}
    }
  }
  #for i in range(3):
  sweep_id = wandb.sweep(sweep_config_1, project = WANDB_PROJECT, entity=WANDB_ENTITY)
  wandb.agent(sweep_id, function=run_wandb, count=cnt)
  sweep_id = wandb.sweep(sweep_config_2, project = WANDB_PROJECT, entity=WANDB_ENTITY)
  wandb.agent(sweep_id, function=run_wandb, count=cnt)
  sweep_id = wandb.sweep(sweep_config_3, project = WANDB_PROJECT, entity=WANDB_ENTITY)
  wandb.agent(sweep_id, function=run_wandb, count=cnt)


if __name__ == '__main__':
  #do_hyperparameter_search_using_wandb("bayes", "cross_entropy", 40)
  do_hyperparameter_search_using_wandb("random", "cross_entropy", 1)




Create sweep with ID: by77q8b8
Sweep URL: https://wandb.ai/cs21m003_cs21d406/mnist/sweeps/by77q8b8


wandb: Agent Starting Run: 5wu15h7j with config:
wandb: 	act_func: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_layers: 3
wandb: 	hidden_nodes: 128
wandb: 	learning_rate: 0.001
wandb: 	loss_func: cross_entropy
wandb: 	optimizer: adam
wandb: 	reg: 0
wandb: 	search_type: random
wandb: 	weight_init: xavier
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


TestAccuracy,▁
TrainingAccuracy,▁▅▄▇█
TrainingLoss,█▄▄▂▁
ValidationAccuracy,▁▆▅██
ValidationLoss,█▃▅▂▁
epoch,▁▃▅▆█
TestAccuracy,0.975
TrainingAccuracy,0.98724
TrainingLoss,0.04244
ValidationAccuracy,0.97067
ValidationLoss,0.09577


Create sweep with ID: 4f5p74b7
Sweep URL: https://wandb.ai/cs21m003_cs21d406/mnist/sweeps/4f5p74b7


wandb: Agent Starting Run: 00ie9go3 with config:
wandb: 	act_func: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_layers: 3
wandb: 	hidden_nodes: 128
wandb: 	learning_rate: 0.001
wandb: 	loss_func: cross_entropy
wandb: 	optimizer: adam
wandb: 	reg: 0
wandb: 	search_type: random
wandb: 	weight_init: xavier
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


TestAccuracy,▁
TrainingAccuracy,▁▅▇██
TrainingLoss,█▄▂▂▁
ValidationAccuracy,▁▆▆█▇
ValidationLoss,█▃▂▁▁
epoch,▁▃▅▆█
TestAccuracy,0.9695
TrainingAccuracy,0.98161
TrainingLoss,0.05913
ValidationAccuracy,0.9675
ValidationLoss,0.11143


Create sweep with ID: 0pztzebd
Sweep URL: https://wandb.ai/cs21m003_cs21d406/mnist/sweeps/0pztzebd


wandb: Agent Starting Run: s2mlxte8 with config:
wandb: 	act_func: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_layers: 3
wandb: 	hidden_nodes: 32
wandb: 	learning_rate: 0.001
wandb: 	loss_func: cross_entropy
wandb: 	optimizer: nadam
wandb: 	reg: 0.0005
wandb: 	search_type: random
wandb: 	weight_init: xavier
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


TestAccuracy,▁
TrainingAccuracy,▁▅▇▇█
TrainingLoss,█▄▂▂▁
ValidationAccuracy,▁▅▆▆█
ValidationLoss,█▄▂▂▁
epoch,▁▃▅▆█
TestAccuracy,0.9619
TrainingAccuracy,0.97131
TrainingLoss,0.20149
ValidationAccuracy,0.9635
ValidationLoss,0.23356


In [ ]:
class digits_mnist(object):
  def __init__(self, MnistData=True):
    if MnistData:
      self.initialize_mnist_data()
    else:
      self.initialize_data()

  def initialize_mnist_data(self):
    (X, self.y), (X_test, self.y_test) = mnist.load_data()
    self.X = np.array([x.flatten() for x in X]) / 255
    self.X_test = np.array([x.flatten() for x in X_test]) / 255

  def initialize_data(self):
    raise NotImplementedError("Please implement this method if you need other dataset.")

  @staticmethod
  def get_loss_function(key):
    mapper = {
      "cross_entropy": CrossEntropy,
      "mse": MeanSquaredError
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_activation_function(key):
    mapper = {
      "sigmoid": Sigmoid,
      "relu": Relu,
      "tanh": Tanh
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_optimizer(key):
    mapper = {
      "sgd": SGD,
      "momentum_gd": MomentumGD,
      "nesterov_gd": NesterovGD,
      "rmsprop": Rmsprop,
      "adam": Adam,
      "nadam": Nadam
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def train(X_train, y_train, params,id,do_val,wandb_log):
    """
    X_train: (batch_size(B), data_size(N))
    y_train: (batch_size(B))
    params: dict(
      batch_size: int,
      epochs: int,
      alpha: float,
      optimizer: One of (SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam),
      hidden_layers_size: list(layer_size),
      act_func: One of (Sigmoid, Relu, Tanh),
      reg: float,
      init: One of (random, xavier),
      loss_func: One of (CrossEntropy, MeanSquaredError)
    )
    """
    print(f"model:{id}")
    if do_val:
      X_train, X_val, y_train, y_val = train_test_split(
          X_train, y_train, test_size=0.1, random_state=10)

    data_size, input_size = X_train.shape
    output_size = 10
    batch_size, epochs = params['batch_size'], params['epochs']
    act_func, loss_func = params['act_func'](), params['loss_func']()
    model = FNN(
      input_size         = input_size,
      output_size        = output_size,
      hidden_layers_size = params['hidden_layers_size'],
      act_func           = act_func,
      reg                = params['reg'],
      init               = params['init'],
      loss_func          = loss_func,
    )
    optimizer = params['optimizer'](model, params['alpha'])
    for ep in range(1, epochs+1):
      ids = np.arange(data_size)
      np.random.shuffle(ids)
      start, end = 0, batch_size
      while end > start:
        x, y = X_train[ids[start:end]], y_train[ids[start:end]]
        optimizer.optimize(x, y)
        start, end = end, min(data_size, end+batch_size)
      # log
      train_loss = loss_func.error(X_train, y_train, model)
      estimate_y_train = digits_mnist.predict(X_train, model)
      train_acc = accuracy(estimate_y_train, y_train)
      val_loss, val_acc = "NotDefined", "NotDefined"
      if do_val:
        val_loss = loss_func.error(X_val, y_val, model)
        estimate_y_val = digits_mnist.predict(X_val, model)
        val_acc = accuracy(estimate_y_val, y_val)
      digits_mnist.logger(train_loss, train_acc, val_loss,val_acc,wandb_log,ep)
    return model

  def logger(train_loss, train_acc, val_loss,val_acc,wandb_log,epoch):
    #print(f"epochs:{epoch}")
    print(f"TrainingLoss: {train_loss}, TrainingAccuracy: {train_acc}")
    print(f"ValidationLoss: {val_loss}, ValidationAccuracy: {val_acc}")
    if wandb_log:
      wandb.log({
        "epoch": epoch,
        "TrainingLoss": train_loss,
        "TrainingAccuracy": train_acc,
        "ValidationLoss": val_loss,
        "ValidationAccuracy": val_acc,  
      })
  @staticmethod
  def predict(X, model):
    """
    X: (batch_size(B), data_size(N))
    return np.array of size: (batch_size(B),)
    """
    prob = model.forward(X)[-1]
    return np.argmax(prob, axis=1)


WANDB_PROJECT = "mnist"
WANDB_ENTITY  = "cs21m003_cs21d406"


def run(epochs,batch_size,hidden_layers,hidden_nodes,learning_rate,optimizer,act_func,reg,weight_init,id,loss_func):
  wandb.init(project=WANDB_PROJECT, entity=WANDB_ENTITY)
  #config = wandb.config
  loss_name = "ce" if loss_func == "cross_entropy" else "mse"
  wandb.run.name=f"e_{epochs}_bs_{batch_size}_hl_{hidden_layers}_hn_{hidden_nodes}_init_{weight_init}_ac_{act_func}_reg_{reg}_ls_{loss_name}_opt_{optimizer}_lr_{learning_rate}_best_mnist"
  hidden_layers_size = [hidden_nodes] * hidden_layers
  runner = digits_mnist()
  params = {
    "batch_size"        : batch_size,
    "epochs"            : epochs,
    "alpha"             : learning_rate,
    "optimizer"         : runner.get_optimizer(optimizer),
    "hidden_layers_size": hidden_layers_size,
    "act_func"          : runner.get_activation_function(act_func),
    "reg"               : reg,
    "init"              : weight_init,
    "loss_func"         : runner.get_loss_function(loss_func),
  }
  model = runner.train(runner.X, runner.y,params,id,do_val=True,wandb_log=True)
  estimate_y_test = runner.predict(runner.X_test, model)
  test_acc = accuracy(estimate_y_test, runner.y_test)
  print(f'test_accuracy:{test_acc}')
  wandb.finish()




if __name__ == '__main__':
  sweep_id = wandb.sweep(project = WANDB_PROJECT, entity=WANDB_ENTITY)
  wandb.agent(sweep_id, function=run(5,32,3,128,0.001,"adam","tanh",0,'xavier',1,"cross_entropy"), count=1)
  #run(5,32,3,128,0.001,"adam","tanh",0,'xavier',1,"cross_entropy")
  #run(5,64,3,128,0.001,"adam","tanh",0,'xavier',2,"cross_entropy")
  #run(5,32,3,32,0.001,"nadam","tanh",0.0005,'xavier',3,"cross_entropy")

TypeError: ignored